In [1]:
from svmutil import *

In [2]:
import pandas as pd
import numpy as np

c:\users\user\appdata\local\programs\python\python36\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
c:\users\user\appdata\local\programs\python\python36\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


In [3]:
# df_data = pd.read_csv('X_train.csv', header=None)
# df_label = pd.read_csv('T_train.csv', header=None)
# labels = df_label.values
# df_data['label'] = labels
# df_data.to_csv('train.csv', index=False)

# # Execute CMD >> python csv2libsvm.py train.csv train.data 784 True

# df_data = pd.read_csv('X_test.csv', header=None)
# df_label = pd.read_csv('T_test.csv', header=None)
# labels = df_label.values
# df_data['label'] = labels
# df_data.to_csv('test.csv', index=False)

# # Execute CMD >> python csv2libsvm.py test.csv test.data 784 True

In [43]:
y_train, X_train = svm_read_problem(r'train.data')
y_test, X_test = svm_read_problem(r'test.data')

C:\Users\USER\Desktop\碩一上\Machine Learning\HW\Machine-Learning-Implementation\HW5\libsvm-3.23\python\commonutil.py:28: ResourceWarning: unclosed file <_io.TextIOWrapper name='train.data' mode='r' encoding='cp950'>
  for i, line in enumerate(open(data_file_name)):
C:\Users\USER\Desktop\碩一上\Machine Learning\HW\Machine-Learning-Implementation\HW5\libsvm-3.23\python\commonutil.py:28: ResourceWarning: unclosed file <_io.TextIOWrapper name='test.data' mode='r' encoding='cp950'>
  for i, line in enumerate(open(data_file_name)):


In [5]:
## Use different kernel functions  (linear, polynomial, and RBF kernels) and have comparison between their performance. 
linear_model =  svm_train(y_train, X_train, '-t 0' )
print('Linear SVM:')
p_label, p_acc, p_val = svm_predict(y_test,  X_test, linear_model)

poly_model =  svm_train(y_train, X_train, '-t 1' )
print('Polynoimal SVM:')
p_label, p_acc, p_val = svm_predict(y_test,  X_test, poly_model)

RBF_model =  svm_train(y_train, X_train, '-t 2' )
print('RBF SVM:')
p_label, p_acc, p_val = svm_predict(y_test,  X_test, RBF_model)

Linear SVM:
Accuracy = 95.08% (2377/2500) (classification)


In [8]:
# use C-SVCdo the grid search
m = svm_train(y_test,  X_test, '-s 0 -v 5')

Cross Validation Accuracy = 95%


In [7]:
help(svm_train)

Help on function svm_train in module svmutil:

svm_train(arg1, arg2=None, arg3=None)
    svm_train(y, x [, options]) -> model | ACC | MSE
    
    y: a list/tuple/ndarray of l true labels (type must be int/double).
    
    x: 1. a list/tuple of l training instances. Feature vector of
          each training instance is a list/tuple or dictionary.
    
       2. an l * n numpy ndarray or scipy spmatrix (n: number of features).
    
    svm_train(prob [, options]) -> model | ACC | MSE
    svm_train(prob, param) -> model | ACC| MSE
    
    Train an SVM model from data (y, x) or an svm_problem prob using
    'options' or an svm_parameter param.
    If '-v' is specified in 'options' (i.e., cross validation)
    either accuracy (ACC) or mean-squared error (MSE) is returned.
    options:
        -s svm_type : set type of SVM (default 0)
            0 -- C-SVC              (multi-class classification)
            1 -- nu-SVC             (multi-class classification)
            2 -- one-class

In [55]:
def create_kernel(x1, x2): # train/train or train/test
    # create dense data for x1 (train)
    max_key=np.max([np.max(v.keys()) for v in x1])
    arr=np.zeros( (len(x1), len(max_key) ))
    for row, vec in enumerate(x1):
        for k, v in vec.items():
            arr[row][k-1]=v
    x1 = arr
    
    #create dense data for x2
    max_key=np.max([np.max(v.keys()) for v in x2])
    arr=np.zeros( (len(x2), len(max_key) ))
    for row, vec in enumerate(x2):
        for k, v in vec.items():
            arr[row][k-1]=v
    x2 = arr
    
    #create a linear kernel matrix 
    k = np.zeros( (len(x2), len(x1)+1) )
    k[:,1:] = np.dot(x2,x1.T)
    k[:,:1] = np.arange(len(x2))[:,np.newaxis]+1
    
    kernel = [list(row) for row in k]
    return kernel


x_train = X_train[:]
kernel = create_kernel(x_train, x_train)
linear_model = svm_train(y_train, kernel, '-t 4')

x_test = X_test[:]
kernel = create_kernel(x_train, x_test)
p_label, p_acc, p_val = svm_predict(y_test, kernel, linear_model)

# #create a linear kernel matrix for the test data
# x_test = np.zeros( (2500, 5001) )
# K_test[:,1:]=np.dot(x[200:],x[:200].T)
# K_test[:,:1]=np.arange(len(x)-200)[:,np.newaxis]+1
'''
#create a linear kernel matrix for the test data

K_test[:,1:]=np.dot(x[200:],x[:200].T)
K_test[:,:1]=np.arange(len(x)-200)[:,np.newaxis]+1

p_label, p_acc, p_val = svm_predict(y[200:],[list(row) for row in K_test], m)
'''


5000
Accuracy = 95.08% (2377/2500) (classification)


'\n#create a linear kernel matrix for the test data\n\nK_test[:,1:]=np.dot(x[200:],x[:200].T)\nK_test[:,:1]=np.arange(len(x)-200)[:,np.newaxis]+1\n\np_label, p_acc, p_val = svm_predict(y[200:],[list(row) for row in K_test], m)\n'